<a href="https://colab.research.google.com/github/Anni1808/MINed_Hackoholics/blob/main/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix, classification_report

# Load the training and test datasets
train_data = pd.read_csv('/content/sample_data/final_merged_dataset.csv')
test_data = pd.read_csv('/content/sample_data/aligned_test.csv')

# Separate features (X) and target (y) for training data
X = train_data.drop('Type', axis=1)  # Features
y = train_data['Type']  # Target

# Remove unnamed columns if they exist
X = X.loc[:, ~X.columns.str.contains('^Unnamed')]
test_data = test_data.loc[:, ~test_data.columns.str.contains('^Unnamed')]

# Select only numeric columns
numeric_cols = X.select_dtypes(include=[np.number]).columns
X_numeric = X[numeric_cols]
X_test_numeric = test_data[numeric_cols]

# Check for and handle infinite values
X_numeric.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test_numeric.replace([np.inf, -np.inf], np.nan, inplace=True)

# Handle missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_numeric = pd.DataFrame(
    imputer.fit_transform(X_numeric),
    columns=X_numeric.columns,
    index=X_numeric.index
)
X_test_numeric = pd.DataFrame(
    imputer.transform(X_test_numeric),
    columns=X_test_numeric.columns,
    index=X_test_numeric.index
)

# Feature scaling
scaler = StandardScaler()
X_scaled = pd.DataFrame(
    scaler.fit_transform(X_numeric),
    columns=X_numeric.columns,
    index=X_numeric.index
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test_numeric),
    columns=X_test_numeric.columns,
    index=X_test_numeric.index
)

# Encode target variable
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split the training data into training and validation subsets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
}

# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Create a KFold object for cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create a GridSearchCV object for hyperparameter tuning
grid_search = GridSearchCV(
    estimator=rf_classifier,
    param_grid=param_grid,
    scoring='accuracy',
    cv=kf,
    verbose=1,
    n_jobs=-1
)

# Fit the model with hyperparameter tuning on the training subset
grid_search.fit(X_train, y_train)

# Get the best model from the grid search
best_rf_model = grid_search.best_estimator_

# Evaluate the model on the validation subset
y_val_pred = best_rf_model.predict(X_val)

# Calculate performance metrics on the validation subset
precision, recall, f1, _ = precision_recall_fscore_support(y_val, y_val_pred, average='weighted')
accuracy = accuracy_score(y_val, y_val_pred)
conf_matrix = confusion_matrix(y_val, y_val_pred)

# Convert target names to strings explicitly
target_names = [str(cls) for cls in le.classes_]

# Print performance metrics
print("\nValidation Set Performance Metrics:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print("\nConfusion Matrix (Validation Data):")
print(conf_matrix)

# Detailed classification report
print("\nDetailed Classification Report (Validation Data):")
print(classification_report(y_val, y_val_pred, target_names=target_names))

# Make predictions on the external test data
y_test_pred_encoded = best_rf_model.predict(X_test_scaled)

# Decode predictions back to original labels
y_test_pred = le.inverse_transform(y_test_pred_encoded)

# Create a DataFrame for predictions
predictions_df = pd.DataFrame({'SHA256': test_data['SHA256'], 'Predictions': y_test_pred})

# Save predictions to a CSV file
predictions_df.to_csv('predictions_rf.csv', index=False)

print("\nPredictions saved to predictions_rf.csv")

Fitting 5 folds for each of 27 candidates, totalling 135 fits


<ipython-input-1-c8412dbd4136>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_numeric.replace([np.inf, -np.inf], np.nan, inplace=True)
<ipython-input-1-c8412dbd4136>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_numeric.replace([np.inf, -np.inf], np.nan, inplace=True)



Validation Set Performance Metrics:
Precision: 0.9603
Recall: 0.9669
F1 Score: 0.9630
Accuracy: 0.9669

Confusion Matrix (Validation Data):
[[40  0  0  0  0  0  0]
 [ 1 37  0  2  0  0  0]
 [ 0  0 39  0  1  0  0]
 [ 0  0  0 39  0  0  1]
 [ 0  0  0  1 39  0  0]
 [ 0  0  0  1  0  0  1]
 [ 0  0  0  0  0  0 40]]

Detailed Classification Report (Validation Data):
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        40
           1       1.00      0.93      0.96        40
           2       1.00      0.97      0.99        40
           3       0.91      0.97      0.94        40
           4       0.97      0.97      0.97        40
           5       0.00      0.00      0.00         2
           6       0.95      1.00      0.98        40

    accuracy                           0.97       242
   macro avg       0.83      0.84      0.83       242
weighted avg       0.96      0.97      0.96       242


Predictions saved to predictions_rf.csv


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m